In [1]:
%run import.ipynb
%run ohe.ipynb

(14800, 41)
(14800, 41)

-----categorical features:
['ACLSWKR', 'AHGA', 'AHSCOL', 'AMARITL', 'AMJIND', 'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE', 'AWKSTAT', 'FILESTAT', 'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1', 'MIGMTR3', 'MIGMTR4', 'MIGSAME', 'MIGSUN', 'PARENT', 'PEFNTVTY', 'PEMNTVTY', 'PENATVTY', 'PRCITSHP', 'TAXINC']

-----numeric features:
['AAGE', 'ADTIND', 'ADTOCC', 'AHRSPAY', 'CAPGAIN', 'CAPLOSS', 'DIVVAL', 'MARSUPWT', 'NOEMP', 'SEOTR', 'VETQVA', 'VETYN', 'WKSWORK']
(14800, 407)


In [2]:
import xgboost as xgb
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

----

In [3]:
df.head()

,AAGE,ADTIND,ADTOCC,AHRSPAY,CAPGAIN,CAPLOSS,DIVVAL,MARSUPWT,NOEMP,SEOTR,VETQVA,VETYN,WKSWORK,ACLSWKR_FederalGovernment,ACLSWKR_LocalGovernment,ACLSWKR_NeverWorked,ACLSWKR_Not_in_universe,ACLSWKR_Private,ACLSWKR_Self-employed-incorporated,ACLSWKR_Self-employed-notIncorporated,ACLSWKR_StateGovernment,ACLSWKR_WithoutPay,ACLSWKR_nan,AHGA_10thgrade,AHGA_11thgrade,AHGA_12thgradenodiploma,AHGA_5thor6thgrade,AHGA_7thand8thgrade,AHGA_9thgrade,AHGA_Associatesdegree-academicprogram,AHGA_Associatesdegree-occup/vocational,AHGA_Bachelorsdegree(BAABBS),AHGA_Children,AHGA_Doctoratedegree(PhDEdD),AHGA_First2nd3rdor4thgrade,AHGA_Highschoolgraduate,AHGA_Lessthan1stgrade,AHGA_Mastersdegree(MAMSMEngMEdMSWMBA),AHGA_Profschooldegree(MDDDSDVMLLBJD),AHGA_Somecollegebutnodegree,AHGA_nan,AHSCOL_Collegeoruniversity,AHSCOL_Highschool,AHSCOL_Not_in_universe,AHSCOL_nan,AMARITL_Divorced,AMARITL_Married-AFspousepresent,AMARITL_Married-civilianspousepresent,AMARITL_Married-spouseabsent,AMARITL_Nevermarried,AMARITL_Separated,AMARITL_Widowed,AMARITL_nan,AMJIND_Agriculture,AMJIND_ArmedForces,AMJIND_Businessandrepairservices,AMJIND_Communications,AMJIND_Construction,AMJIND_Education,AMJIND_Entertainment,AMJIND_Financeinsuranceandrealestate,AMJIND_Forestryandfisheries,AMJIND_Hospitalservices,AMJIND_Manufacturing-durablegoods,AMJIND_Manufacturing-nondurablegoods,AMJIND_Medicalexcepthospital,AMJIND_Mining,AMJIND_Not_in_universeorchildren,AMJIND_Otherprofessionalservices,AMJIND_PersonalservicesexceptprivateHH,AMJIND_Privatehouseholdservices,AMJIND_Publicadministration,AMJIND_Retailtrade,AMJIND_Socialservices,AMJIND_Transportation,AMJIND_Utilitiesandsanitaryservices,AMJIND_Wholesaletrade,AMJIND_nan,AMJOCC_Admsupportincludingclerical,AMJOCC_ArmedForces,AMJOCC_Executiveadminandmanagerial,AMJOCC_Farmingforestryandfishing,AMJOCC_Handlersequipcleanersetc,AMJOCC_Machineoperatorsassmblrs&inspctrs,AMJOCC_Not_in_universe,AMJOCC_Otherservice,AMJOCC_Precisionproductioncraft&repair,AMJOCC_Privatehouseholdservices,AMJOCC_Professionalspecialty,AMJOCC_Protectiveservices,AMJOCC_Sales,AMJOCC_Techniciansandrelatedsupport,AMJOCC_Transportationandmaterialmoving,AMJOCC_nan,ARACE_AmerIndianAleutorEskimo,ARACE_AsianorPacificIslander,ARACE_Black,ARACE_Other,ARACE_White,ARACE_nan,AREORGN_Allother,AREORGN_CentralorSouthAmerican,AREORGN_Chicano,AREORGN_Cuban,AREORGN_Donotknow,AREORGN_Mexican(Mexicano),AREORGN_Mexican-American,AREORGN_OtherSpanish,AREORGN_PuertoRican,AREORGN_nan,ASEX_Female,ASEX_Male,ASEX_nan,AUNMEM_No,AUNMEM_Not_in_universe,AUNMEM_Yes,AUNMEM_nan,AUNTYPE_Jobleaver,AUNTYPE_Jobloser-onlayoff,AUNTYPE_Newentrant,AUNTYPE_Not_in_universe,AUNTYPE_Otherjobloser,AUNTYPE_Re-entrant,AUNTYPE_nan,AWKSTAT_ChildrenorArmedForces,AWKSTAT_nan,FILESTAT_Headofhousehold,FILESTAT_Jointboth65+,FILESTAT_Jointbothunder65,FILESTAT_Jointoneunder65&one65+,FILESTAT_Nonfiler,FILESTAT_Single,FILESTAT_nan,GRINREG_Abroad,GRINREG_Midwest,GRINREG_Northeast,GRINREG_Not_in_universe,GRINREG_South,GRINREG_West,GRINREG_nan,GRINST_Abroad,GRINST_Alabama,GRINST_Alaska,GRINST_Arizona,GRINST_Arkansas,GRINST_California,GRINST_Colorado,GRINST_Connecticut,GRINST_Delaware,GRINST_DistrictofColumbia,GRINST_Florida,GRINST_Georgia,GRINST_Idaho,GRINST_Illinois,GRINST_Indiana,GRINST_Iowa,GRINST_Kansas,GRINST_Kentucky,GRINST_Louisiana,GRINST_Maine,GRINST_Maryland,GRINST_Massachusetts,GRINST_Michigan,GRINST_Minnesota,GRINST_Mississippi,GRINST_Missouri,GRINST_Montana,GRINST_Nebraska,GRINST_Nevada,GRINST_NewHampshire,GRINST_NewJersey,GRINST_NewMexico,GRINST_NewYork,GRINST_NorthCarolina,GRINST_NorthDakota,GRINST_Not_in_universe,GRINST_Ohio,GRINST_Oklahoma,GRINST_Oregon,GRINST_Pennsylvania,GRINST_SouthCarolina,GRINST_SouthDakota,GRINST_Tennessee,GRINST_Texas,GRINST_Utah,GRINST_Vermont,GRINST_Virginia,GRINST_WestVirginia,GRINST_Wisconsin,GRINST_Wyoming,GRINST_nan,HHDFMX_Child18+evermarrNotinasubfamily,HHDFMX_Child18+evermarrRPofsubfamily,HHDFMX_Child18+nevermarrNotinasubfamily,HHDFMX_Child18+nevermarrRPofsubfamily,HHDFMX_Child18+spouseofsub

In [4]:
vars_to_drop = ['AWKSTAT_ChildrenorArmedForces', 'AWKSTAT_nan', 'WKSWORK']
df.drop(vars_to_drop, axis=1, inplace=True)

In [5]:
df.fillna(0, inplace=True)

------

In [6]:
X = df[:10000]
X_test = df[10000:]
print(X.shape)
print(X_test.shape)

(10000, 404)
(4800, 404)


----

In [7]:
y.value_counts()

-50000    5000
50000+    5000
Name: CLASS, dtype: int64

In [8]:
y.replace({'-50000' : 0, '50000+' : 1}, inplace=True)

----

In [9]:
dtrain = xgb.DMatrix(X, y, missing=0.0)
dtest = xgb.DMatrix(X_test)

In [10]:
param = { 'objective' : "binary:logistic" }
num_round = 100
bst = xgb.train(param, dtrain, num_round)

In [11]:
pred = bst.predict(dtest)

In [12]:
pred = pred.round()

In [13]:
filename = 'submissions/xgb_' + datetime.now().strftime(format = '%d%m_%H%M') + '.csv'
pd.DataFrame({'ID':ids, 'Prediction': pred}).to_csv(filename, index=False, sep=',')